In [1]:
import os

from summarizer.sbert import SBertSummarizer
import torch

from tqdm.notebook import tqdm

from db.DocumentRepository import DocumentRepository

/Users/danielwps/Documents/Private/Github/Project_MSE/venv/lib/python3.11/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


In [2]:
os.system("""
    docker compose down;
    docker compose up -d --build db;
    sleep 15;
    """)

 Container project_mse-db-1  Stopping
 Container project_mse-db-1  Stopped
 Container project_mse-db-1  Removing
 Container project_mse-db-1  Removed
 Network project_mse_default  Removing
 Network project_mse_default  Removed


#0 building with "desktop-linux" instance using docker driver

#1 [db internal] load build definition from Dockerfile
#1 transferring dockerfile: 159B done
#1 DONE 0.0s

#2 [db internal] load metadata for docker.io/library/postgres:latest
#2 DONE 0.0s

#3 [db internal] load .dockerignore
#3 transferring context: 2B done
#3 DONE 0.0s

#4 [db 1/2] FROM docker.io/library/postgres:latest
#4 DONE 0.0s

#5 [db internal] load build context
#5 transferring context: 135.94MB 1.5s done
#5 DONE 1.5s

#4 [db 1/2] FROM docker.io/library/postgres:latest
#4 CACHED

#6 [db 2/2] COPY dump.sql /docker-entrypoint-initdb.d/
#6 DONE 0.4s

#7 [db] exporting to image
#7 exporting layers
#7 exporting layers 0.6s done
#7 writing image sha256:e2c78629fa119c48a2dd096074413a38b4d92e1f0ba93da9531ef0189af74c00 done
#7 naming to docker.io/library/project_mse-db done
#7 DONE 0.6s


 Network project_mse_default  Creating
 Network project_mse_default  Created
 Container project_mse-db-1  Creating
 Container project_mse-db-1  Created
 Container project_mse-db-1  Starting
 Container project_mse-db-1  Started


0

In [12]:
documentRepository = DocumentRepository()
tokenizer = documentRepository.tokenizer

SC: Connected to the db. Now you can go and build the best search engine around!


In [4]:
summarizer = SBertSummarizer('paraphrase-MiniLM-L6-v2')
summarizer

In [5]:
all_docs = documentRepository.loadAllDocuments()
len(all_docs)

9147

In [6]:
print(all_docs[0].enc_text)
print(all_docs[0].summary)
print(all_docs[0].external_links)
print(all_docs[0].internal_links)

tensor([])
None
[]
[]


In [7]:
print(len(all_docs))
number_none_enc_text = [doc.enc_text for doc in all_docs].count(None)
number_none_summary = [doc.summary for doc in all_docs].count(None)
number_none_external_links = [doc.external_links == [] for doc in all_docs].count(False)
number_none_internal_links = [doc.internal_links == [] for doc in all_docs].count(False)
print(f"Does the Database pass the following?\n"
      f"Contains encoded text for every doc: {number_none_enc_text == 0}\n"
      f"Contains summary for every doc: {number_none_summary == 0}\n"
      f"Contains no more external links for every doc: {number_none_external_links == 0}\n"
      f"Contains no internal links for every doc: {number_none_internal_links == 0}\n")

9147
Does the Database pass the following?
Contains encoded text for every doc: True
Contains summary for every doc: False
Contains no more external links for every doc: False
Contains no internal links for every doc: False



If the test is giving you "False" for any of them, you should go and execute the other cells as well. If you got "True", you are done with this notebook already. :)

In [8]:
# 1. Step: Encode the text
for doc in tqdm(all_docs):
    text = doc.page_text
    
    encode = tokenizer.encode(text, add_special_tokens=False)
    doc_vec = torch.tensor(encode)
    doc.enc_text = doc_vec

  0%|          | 0/9147 [00:00<?, ?it/s]

Token indices sequence length is longer than the specified maximum sequence length for this model (5958 > 512). Running this sequence through the model will result in indexing errors


In [9]:
# 2. Step: Summarize the text
for doc in tqdm(all_docs):
    text = doc.page_text
    
    summary = summarizer(text, min_length=60, num_sentences=3, max_length=300)
    doc.summary = summary

  0%|          | 0/9147 [00:00<?, ?it/s]

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
/Users/danielwps/Documents/Private/Github/Project_MSE/venv/lib/python3.11/site-packages/threadpoolctl.py:1214: RuntimeWarning: 
Found Intel OpenMP ('libiomp') and LLVM OpenMP ('libomp') loaded at
the same time. Both libraries are known to be incompatible and this
can cause random crashes or deadlocks on Linux when loaded in the
same Python program.
Using threadpoolctl may cause crashes or deadlocks. For more
information and possible workarounds, please see
    https://github.com/joblib/threadpoolctl/blob/master/multiple_openmp.md

  warnings.warn(msg, RuntimeWarning)
/Users/danielwps/Documents/Private/Github/Project_MSE/venv/lib/python3.11/site-packages/sklearn/base.p

In [10]:
# Step 3: delete entries internal and external links because we don't use them during inference
for doc in tqdm(all_docs):
    doc.internal_links = []
    doc.external_links = []

  0%|          | 0/9147 [00:00<?, ?it/s]

In [13]:
for doc in tqdm(all_docs):
    documentRepository.updateDocument(doc)

  0%|          | 0/9147 [00:00<?, ?it/s]

SC: Updated document.
SC: Updated document.
SC: Updated document.
SC: Updated document.
SC: Updated document.
SC: Updated document.
SC: Updated document.
SC: Updated document.
SC: Updated document.
SC: Updated document.
SC: Updated document.
SC: Updated document.
SC: Updated document.
SC: Updated document.
SC: Updated document.
SC: Updated document.
SC: Updated document.
SC: Updated document.
SC: Updated document.
SC: Updated document.
SC: Updated document.
SC: Updated document.
SC: Updated document.
SC: Updated document.
SC: Updated document.
SC: Updated document.
SC: Updated document.
SC: Updated document.
SC: Updated document.
SC: Updated document.
SC: Updated document.
SC: Updated document.
SC: Updated document.
SC: Updated document.
SC: Updated document.
SC: Updated document.
SC: Updated document.
SC: Updated document.
SC: Updated document.
SC: Updated document.
SC: Updated document.
SC: Updated document.
SC: Updated document.
SC: Updated document.
SC: Updated document.
SC: Update

In [14]:
new_all_docs = documentRepository.loadAllDocuments()
print(len(all_docs))
number_none_enc_text = [doc.enc_text for doc in new_all_docs].count(None)
number_none_summary = [doc.summary for doc in new_all_docs].count(None)
number_none_external_links = [doc.external_links == [] for doc in new_all_docs].count(False)
number_none_internal_links = [doc.internal_links == [] for doc in new_all_docs].count(False)
print(f"Does the Database pass the following?\n"
      f"Contains encoded text for every doc: {number_none_enc_text == 0}\n"
      f"Contains summary for every doc: {number_none_summary == 0}\n"
      f"Contains no more external links for every doc: {number_none_external_links == 0}\n"
      f"Contains no internal links for every doc: {number_none_internal_links == 0}\n")

9147
Does the Database pass the following?
Contains encoded text for every doc: True
Contains summary for every doc: True
Contains no more external links for every doc: True
Contains no internal links for every doc: True



In [15]:
if number_none_enc_text == 0 and number_none_summary == 0 and number_none_external_links == 0 and number_none_internal_links == 0:
    documentRepository.overwrite_dump()

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


SC: Successfully overwritten the old dump. Now you only need to push it to the repository!


**Deprecated:**
Our current index is too big for ranking BM25 with all of them during inference. Let's shorten it

In [ ]:
# cutted_docs = new_all_docs[:200]
# documentRepository.deleteAllDocuments()
# documentRepository.saveAllDocuments(cutted_docs)
# saved_docs = documentRepository.loadAllDocuments()
# len(saved_docs)